In [5]:
# Import library
import pandas as pd
import numpy as np
import zipfile

# Import data from .zip file
zf = zipfile.ZipFile('2015_Street_Tree_Census_-_Tree_Data.zip')
df_tree = pd.read_csv(zf.open('2015_Street_Tree_Census_-_Tree_Data.csv'))
df_tree.head()

,tree_id,block_id,created_at,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,spc_common,...,boro_ct,state,latitude,longitude,x_sp,y_sp,council district,census tract,bin,bbl
0,180683,348711,2015-08-27,3,0,OnCurb,Alive,Fair,Acer rubrum,red maple,...,4073900,New York,40.723092,-73.844215,1027431.148,202756.7687,29.0,739.0,4052307.0,4.022210e+09
1,200540,315986,2015-09-03,21,0,OnCurb,Alive,Fair,Quercus palustris,pin oak,...,4097300,New York,40.794111,-73.818679,1034455.701,228644.8374,19.0,973.0,4101931.0,4.044750e+09
2,204026,218365,2015-09-05,3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.717581,-73.936608,1001822.831,200716.8913,34.0,449.0,3338310.0,3.028870e+09
3,204337,217969,2015-09-05,10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,honeylocust,...,3044900,New York,40.713537,-73.934456,1002420.358,199244.2531,34.0,449.0,3338342.0,3.029250e+09
4,189565,223043,2015-08-30,21,0,OnCurb,Alive,Good,Tilia americana,American linden,...,3016500,New York,40.666778,-73.975979,990913.775,182202.4260,39.0,165.0,3025654.0,3.010850e+09


In [2]:
# Change data type for datetime
df_tree["created_at"] = pd.to_datetime(df_tree["created_at"], format='%Y-%m-%d')
df_tree.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 45 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   tree_id           683788 non-null  int64         
 1   block_id          683788 non-null  int64         
 2   created_at        683788 non-null  datetime64[ns]
 3   tree_dbh          683788 non-null  int64         
 4   stump_diam        683788 non-null  int64         
 5   curb_loc          683788 non-null  object        
 6   status            683788 non-null  object        
 7   health            652172 non-null  object        
 8   spc_latin         652169 non-null  object        
 9   spc_common        652169 non-null  object        
 10  steward           652173 non-null  object        
 11  guards            652172 non-null  object        
 12  sidewalk          652172 non-null  object        
 13  user_type         683788 non-null  object        
 14  prob

In [3]:
# Source: https://abhisheksingh007226.medium.com/how-to-compress-csv-file-efficiently-in-just-25-lines-of-code-4e974b60b2b8
# Compress numerical features by adjusting floating types
def compress(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_tree = compress(df_tree)
df_tree.info(memory_usage='deep')

Mem. usage decreased to 168.24 Mb (28.3% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683788 entries, 0 to 683787
Data columns (total 45 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   tree_id           683788 non-null  int32         
 1   block_id          683788 non-null  int32         
 2   created_at        683788 non-null  datetime64[ns]
 3   tree_dbh          683788 non-null  int16         
 4   stump_diam        683788 non-null  int16         
 5   curb_loc          683788 non-null  object        
 6   status            683788 non-null  object        
 7   health            652172 non-null  object        
 8   spc_latin         652169 non-null  object        
 9   spc_common        652169 non-null  object        
 10  steward           652173 non-null  object        
 11  guards            652172 non-null  object        
 12  sidewalk          652172 non-null  object        
 13  user_ty

In [4]:
# Take a sample of the data set and save file
df_tree_mini = df_tree.sample(n=100000, random_state=99)
df_tree_mini.to_csv('2015_Street_Tree_Census_-_Tree_Data_sample.csv', index=False)